# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8db7fc9850>
├── 'a' --> tensor([[ 0.8739, -1.0014,  0.3269],
│                   [ 1.1600, -1.0678,  0.0530]])
└── 'x' --> <FastTreeValue 0x7f8db7fc9ca0>
    └── 'c' --> tensor([[-1.3067,  0.6583,  0.5378,  1.3737],
                        [-0.2839,  1.0994,  0.9962,  0.8344],
                        [-1.7274, -1.2973,  1.9807, -0.3196]])

In [4]:
t.a

tensor([[ 0.8739, -1.0014,  0.3269],
        [ 1.1600, -1.0678,  0.0530]])

In [5]:
%timeit t.a

64 ns ± 0.0649 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8db7fc9850>
├── 'a' --> tensor([[-0.4581, -1.1743,  0.2467],
│                   [ 2.0726,  0.3882, -1.2540]])
└── 'x' --> <FastTreeValue 0x7f8db7fc9ca0>
    └── 'c' --> tensor([[-1.3067,  0.6583,  0.5378,  1.3737],
                        [-0.2839,  1.0994,  0.9962,  0.8344],
                        [-1.7274, -1.2973,  1.9807, -0.3196]])

In [7]:
%timeit t.a = new_value

65.3 ns ± 0.0559 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8739, -1.0014,  0.3269],
               [ 1.1600, -1.0678,  0.0530]]),
    x: Batch(
           c: tensor([[-1.3067,  0.6583,  0.5378,  1.3737],
                      [-0.2839,  1.0994,  0.9962,  0.8344],
                      [-1.7274, -1.2973,  1.9807, -0.3196]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8739, -1.0014,  0.3269],
        [ 1.1600, -1.0678,  0.0530]])

In [11]:
%timeit b.a

55.4 ns ± 0.0189 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7400,  0.3281,  0.2407],
               [-1.5717, -0.5963,  2.2515]]),
    x: Batch(
           c: tensor([[-1.3067,  0.6583,  0.5378,  1.3737],
                      [-0.2839,  1.0994,  0.9962,  0.8344],
                      [-1.7274, -1.2973,  1.9807, -0.3196]]),
       ),
)

In [13]:
%timeit b.a = new_value

476 ns ± 0.107 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

872 ns ± 1.62 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 28.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 203 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 934 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8d8c406d60>
├── 'a' --> tensor([[[ 0.8739, -1.0014,  0.3269],
│                    [ 1.1600, -1.0678,  0.0530]],
│           
│                   [[ 0.8739, -1.0014,  0.3269],
│                    [ 1.1600, -1.0678,  0.0530]],
│           
│                   [[ 0.8739, -1.0014,  0.3269],
│                    [ 1.1600, -1.0678,  0.0530]],
│           
│                   [[ 0.8739, -1.0014,  0.3269],
│                    [ 1.1600, -1.0678,  0.0530]],
│           
│                   [[ 0.8739, -1.0014,  0.3269],
│                    [ 1.1600, -1.0678,  0.0530]],
│           
│                   [[ 0.8739, -1.0014,  0.3269],
│                    [ 1.1600, -1.0678,  0.0530]],
│           
│                   [[ 0.8739, -1.0014,  0.3269],
│                    [ 1.1600, -1.0678,  0.0530]],
│           
│                   [[ 0.8739, -1.0014,  0.3269],
│                    [ 1.1600, -1.0678,  0.0530]]])
└── 'x' --> <FastTreeValue 0x7f8d8c3fc070>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 45.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8d8c406cd0>
├── 'a' --> tensor([[ 0.8739, -1.0014,  0.3269],
│                   [ 1.1600, -1.0678,  0.0530],
│                   [ 0.8739, -1.0014,  0.3269],
│                   [ 1.1600, -1.0678,  0.0530],
│                   [ 0.8739, -1.0014,  0.3269],
│                   [ 1.1600, -1.0678,  0.0530],
│                   [ 0.8739, -1.0014,  0.3269],
│                   [ 1.1600, -1.0678,  0.0530],
│                   [ 0.8739, -1.0014,  0.3269],
│                   [ 1.1600, -1.0678,  0.0530],
│                   [ 0.8739, -1.0014,  0.3269],
│                   [ 1.1600, -1.0678,  0.0530],
│                   [ 0.8739, -1.0014,  0.3269],
│                   [ 1.1600, -1.0678,  0.0530],
│                   [ 0.8739, -1.0014,  0.3269],
│                   [ 1.1600, -1.0678,  0.0530]])
└── 'x' --> <FastTreeValue 0x7f8d08b53190>
    └── 'c' --> tensor([[-1.3067,  0.6583,  0.5378,  1.3737],
                        [-0.2839,  1.0994,  0.9962,  0.8344],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 49.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.3067,  0.6583,  0.5378,  1.3737],
                       [-0.2839,  1.0994,  0.9962,  0.8344],
                       [-1.7274, -1.2973,  1.9807, -0.3196]],
              
                      [[-1.3067,  0.6583,  0.5378,  1.3737],
                       [-0.2839,  1.0994,  0.9962,  0.8344],
                       [-1.7274, -1.2973,  1.9807, -0.3196]],
              
                      [[-1.3067,  0.6583,  0.5378,  1.3737],
                       [-0.2839,  1.0994,  0.9962,  0.8344],
                       [-1.7274, -1.2973,  1.9807, -0.3196]],
              
                      [[-1.3067,  0.6583,  0.5378,  1.3737],
                       [-0.2839,  1.0994,  0.9962,  0.8344],
                       [-1.7274, -1.2973,  1.9807, -0.3196]],
              
                      [[-1.3067,  0.6583,  0.5378,  1.3737],
                       [-0.2839,  1.0994,  0.9962,  0.8344],
                       [-1.7274, -1.2973,  1.9807, -0.3196]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 167 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.3067,  0.6583,  0.5378,  1.3737],
                      [-0.2839,  1.0994,  0.9962,  0.8344],
                      [-1.7274, -1.2973,  1.9807, -0.3196],
                      [-1.3067,  0.6583,  0.5378,  1.3737],
                      [-0.2839,  1.0994,  0.9962,  0.8344],
                      [-1.7274, -1.2973,  1.9807, -0.3196],
                      [-1.3067,  0.6583,  0.5378,  1.3737],
                      [-0.2839,  1.0994,  0.9962,  0.8344],
                      [-1.7274, -1.2973,  1.9807, -0.3196],
                      [-1.3067,  0.6583,  0.5378,  1.3737],
                      [-0.2839,  1.0994,  0.9962,  0.8344],
                      [-1.7274, -1.2973,  1.9807, -0.3196],
                      [-1.3067,  0.6583,  0.5378,  1.3737],
                      [-0.2839,  1.0994,  0.9962,  0.8344],
                      [-1.7274, -1.2973,  1.9807, -0.3196],
                      [-1.3067,  0.6583,  0.5378,  1.3737],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

302 µs ± 1.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
